In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/heart-disease-prediction-using-logistic-regression/framingham.csv


In [2]:
data = pd.read_csv('/kaggle/input/heart-disease-prediction-using-logistic-regression/framingham.csv')
data.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [3]:
data['TenYearCHD'].value_counts()

0    3594
1     644
Name: TenYearCHD, dtype: int64

In [4]:
data.isna().sum()

male                 0
age                  0
education          105
currentSmoker        0
cigsPerDay          29
BPMeds              53
prevalentStroke      0
prevalentHyp         0
diabetes             0
totChol             50
sysBP                0
diaBP                0
BMI                 19
heartRate            1
glucose            388
TenYearCHD           0
dtype: int64

In [5]:
data.dropna(axis=0,inplace=True)
data['TenYearCHD'].value_counts()

0    3099
1     557
Name: TenYearCHD, dtype: int64

### Fill in the Missing Values : Mode

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3656 entries, 0 to 4237
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   male             3656 non-null   int64  
 1   age              3656 non-null   int64  
 2   education        3656 non-null   float64
 3   currentSmoker    3656 non-null   int64  
 4   cigsPerDay       3656 non-null   float64
 5   BPMeds           3656 non-null   float64
 6   prevalentStroke  3656 non-null   int64  
 7   prevalentHyp     3656 non-null   int64  
 8   diabetes         3656 non-null   int64  
 9   totChol          3656 non-null   float64
 10  sysBP            3656 non-null   float64
 11  diaBP            3656 non-null   float64
 12  BMI              3656 non-null   float64
 13  heartRate        3656 non-null   float64
 14  glucose          3656 non-null   float64
 15  TenYearCHD       3656 non-null   int64  
dtypes: float64(9), int64(7)
memory usage: 485.6 KB


* Demographic:
* • Sex: male or female(Nominal)
* • Age: Age of the patient;(Continuous - Although the recorded ages have been truncated to whole numbers, the concept of age is continuous)
* Behavioral
* • Current Smoker: whether or not the patient is a current smoker (Nominal)
* • Cigs Per Day: the number of cigarettes that the person smoked on average in one day.(can be considered continuous as one can have any number of cigarettes, even half a cigarette.)
* Medical( history)
* • BP Meds: whether or not the patient was on blood pressure medication (Nominal)
* • Prevalent Stroke: whether or not the patient had previously had a stroke (Nominal)
* • Prevalent Hyp: whether or not the patient was hypertensive (Nominal)
* • Diabetes: whether or not the patient had diabetes (Nominal)
* Medical(current)
* • Tot Chol: total cholesterol level (Continuous)
* • Sys BP: systolic blood pressure (Continuous)
* • Dia BP: diastolic blood pressure (Continuous)
* • BMI: Body Mass Index (Continuous)
* • Heart Rate: heart rate (Continuous - In medical research, variables such as heart rate though in fact discrete, yet are considered continuous because of large number of possible values.)
* • Glucose: glucose level (Continuous)
* Predict variable (desired target)
* • 10 year risk of coronary heart disease CHD (binary: “1”, means “Yes”, “0” means “No”)

# Exploratory Data Analysis

In [7]:
import matplotlib.pyplot as plt
#Gender vs TenYearCHD
x=pd.crosstab(data.male,data.TenYearCHD,margins=True)/data.shape[0]
x

TenYearCHD,0,1,All
male,,,
0,0.487965,0.068381,0.556346
1,0.359683,0.083972,0.443654
All,0.847648,0.152352,1.000000


In [8]:
data.shape[0]

3656

* The dataset of 3656 records has 55.6% percentage data on women and 44.4% data  on men. Out these 7% of women,8.3% of men have a risk of getting coronary heart disease.

* Performed Stratified sampling because of class imbalance.

In [9]:
from sklearn.model_selection import train_test_split
y=data.pop('TenYearCHD')
X=data
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=1,shuffle=True,stratify=y)

In [10]:
from sklearn.preprocessing import StandardScaler
ss=StandardScaler()
X_train =ss.fit_transform(X_train)
X_test =ss.fit_transform(X_test)

In [11]:

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier

lr=LogisticRegression(C=2,solver='liblinear')
lr.fit(X_train,y_train)
lr_y_pred=lr.predict(X_test)
print("The accuracy is %f" %accuracy_score(y_test,lr_y_pred))
confusion_matrix(y_test,lr_y_pred)

The accuracy is 0.846827


array([[766,   9],
       [131,   8]])

# SVC

In [12]:
svc=SVC(C=1,kernel='linear')
svc.fit(X_train,y_train)
svc_y_pred=svc.predict(X_test)
print("The accuracy is %f" %accuracy_score(y_test,svc_y_pred))
confusion_matrix(y_test,svc_y_pred)

The accuracy is 0.847921


array([[775,   0],
       [139,   0]])

The SVC doesn't work for us as it's not giving us any true positives and is probably using high threshold as decision boundary which is expected, compared to logistic regression

# Decision Tree

In [13]:
dc = DecisionTreeClassifier()
dc.fit(X_train,y_train)
y_pred=dc.predict(X_test)
print("The accuracy is %f" %accuracy_score(y_test,y_pred))
confusion_matrix(y_test,y_pred)

The accuracy is 0.765864


array([[664, 111],
       [103,  36]])

High Number of misclassifications from the decision decision which makes it worse than the SVC, and Logistic Regression.(FP=120,FN=97)

# Random Forest

In [14]:
rf=RandomForestClassifier(max_depth=100,criterion='entropy',max_features='log2')
rf.fit(X_train,y_train)
rf_y_pred=rf.predict(X_test)
print("The accuracy is %f" %accuracy_score(y_test,rf_y_pred))
confusion_matrix(y_test,rf_y_pred)

The accuracy is 0.846827


array([[768,   7],
       [133,   6]])

# XGB 

In [15]:
xg=  XGBClassifier(n_estimators=50,booster='dart',learning_rate=0.07)
xg.fit(X_train,y_train)
xg_y_pred = xg.predict(X_test)
print("The accuracy is %f" %accuracy_score(y_test,xg_y_pred))
confusion_matrix(y_test,xg_y_pred)

The accuracy is 0.844639


array([[762,  13],
       [129,  10]])

In [16]:
pd.DataFrame(xg_y_pred).to_csv('submission')

The models are gonna get better predictions with hypertuning for sure, please look at the documentations are try to get the maximum. I hope there was something to learn from my notebook and Look forward to solving more problems! thank you, if you found it useful, please vote!